In [5]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = 'perth-sagemaker'
#outputpath='s3://perth-sagemaker/Data_Repository/saeid-test/Models'
#training_path = 's3://perth-sagemaker/Data_Repository/saeid-test/QC-2-FJ'

#outputpath='s3://perth-sagemaker/Data_Repository/CathX_Woodside/Models'
#training_path = 's3://perth-sagemaker/Data_Repository/CathX_Woodside/BRU_Original_With_Test'

outputpath='s3://perth-sagemaker/Data_Repository/CathX_Woodside/Models'
training_path = 's3://perth-sagemaker/Data_Repository/CathX_Woodside/AF3_Original_With_Test_Resized'



role = sagemaker.get_execution_role()

## Script

Show the entry point script inline for reference

In [7]:
!pygmentize Source/PipeImagTransferLearning.py

from __future__ import print_function, division
import sagemaker_containers
import logging
import sys
import PIL
import argparse
import json
import os
import torch
import numpy as np
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
from collections import OrderedDict
import boto3
import tarfile
import datetime
import torchnet as tnt
import math

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))
validation_loss=[]
min_val_loss=9999
train_accuracy=[]
validation_accuracy=[]
# MEAN = [0.413, 0.413, 0.413]
# STD = [0.159, 0.161, 0.158]
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

def _average_gradients(model):
    # Gradient averaging.
    size = float(dist.get_world_size())
    for param in mode

        self.batch_size = batch_size
        self.training_dir = training_dir

    def get_data_loader(self):
        
        logger.info("Get train data loader")

        data_transform = transforms.Compose([
            transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=None, 
                                    shear=None, resample=PIL.Image.BILINEAR, fillcolor=0),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.25, hue=0.1),
            transforms.RandomHorizontalFlip(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN, std=STD)
        ])

        dataset = ImageFolderWithPaths(self.training_dir, data_transform)
        print(dataset)
        print(dataset.classes)

        weights = Make_weights_for_balanced_classes(
                    dataset.imgs, len(dataset.classes)).get_weights()

        sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(wei

## Training

### Create PyTorch estimator

Instantiate a sagemaker estimator object and pass it the entry point script and path to save the model

This is also where you may define non-default hyperparameters found in the entry point script

In [8]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="PipeImagTransferLearning.py",
                    output_path=outputpath,
                    base_job_name='Images-resnet50-AF3',
                    source_dir='Source',
                    train_volume_size=20,
                    role=role,
                    framework_version='1.0.0', 
                    train_instance_count=1,
                    train_max_run=(5*24*60*60),
                    #train_instance_type='ml.p3.xlarge',
                    train_instance_type='ml.p2.8xlarge',
                    hyperparameters={
                        'batch_size': 64,
                        'lr': 0.01,
                        'lr_step': 10,
                        'data_location':training_path,
                        'epochs':20,
                        'mean': [0.485, 0.456, 0.406],
                        'std':[0.229, 0.224, 0.225],
                        'no_of_classes':2,             
                        'model_def':'resnet50'
                    }) # hyperparameters gets replaced with a config file using SGD with warm start.

In [9]:
estimator.fit({'training': training_path})

INFO:sagemaker:Creating training-job with name: Images-resnet50-AF3-2019-05-10-03-47-41-760


2019-05-10 03:47:42 Starting - Starting the training job...
2019-05-10 03:47:45 Starting - Launching requested ML instances.........
2019-05-10 03:49:24 Starting - Preparing the instances for training......
2019-05-10 03:50:40 Downloading - Downloading input data
2019-05-10 03:50:40 Training - Downloading the training image......
2019-05-10 03:51:24 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-05-10 03:51:26,172 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-05-10 03:51:26,250 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-05-10 03:51:26,252 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-05-10 03:51:26,610 sagemaker-containers INFO     Module PipeImagTransferLearning does not provide a setup.py. 
Generating setup.py
2019-05-1

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
2019-05-10 03:51:39,591 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_pytorch_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "no_of_classes": 2,
        "std": [
            0.229,
            0.224,
            0.225
        ],
        "batch_size": 64,
        "model_def": "resnet50",
        "lr": 0.01,
        "data_location": "s3://perth-sagemaker/Data_Repository/CathX_Woodside/AF3_Original_With_Test_Resized",
        "mean": [
            0.485,
            0.456,
            0.406
        ],
        "epochs": 20,
        "lr_step": 10
    },
    "input_config_dir": "/opt/ml/i

Model on cuda: True
Model on cuda: True
Epoch 1/20
Epoch 1/20
----------
----------
NCCL version 2.3.7+cuda9.0
Train accuracy: [tensor(0.5200, device='cuda:0', dtype=torch.float64)]
Train accuracy: [tensor(0.5200, device='cuda:0', dtype=torch.float64)]
Train Loss: 0.6931
Train Loss: 0.6931
Train Acc: 0.5200
Train Acc: 0.5200
Validation accuracy: [tensor(0.5726, device='cuda:0', dtype=torch.float64)]
Validation accuracy: [tensor(0.5726, device='cuda:0', dtype=torch.float64)]
Val Loss: 0.6844
Val Loss: 0.6844
Val Acc: 0.5726
Val Acc: 0.5726


Learning rate: 0.0100
Learning rate: 0.0100
Confusion matrix: 
[[24 30]
 [23 47]]
Confusion matrix: 
[[24 30]
 [23 47]]






Incorrect prediction paths: [('/opt/ml/input/data/training/validation/Field_Joint/AF3 - 000481 - KP-0.0140 - 2018-09-20_11-28-59#1_Resized_224.jpg', array([-0.00378057,  0.08708327], dtype=float32)), ('/opt/ml/input/data/training/validation/Field_Joint/AF3 - 001368 - KP0.1595 - 2018-09-20_11-33-55#1_Resized_224.jpg', array([0

Train accuracy: [tensor(0.5200, device='cuda:0', dtype=torch.float64), tensor(0.5400, device='cuda:0', dtype=torch.float64), tensor(0.4800, device='cuda:0', dtype=torch.float64), tensor(0.5700, device='cuda:0', dtype=torch.float64), tensor(0.5300, device='cuda:0', dtype=torch.float64)]
Train accuracy: [tensor(0.5200, device='cuda:0', dtype=torch.float64), tensor(0.5400, device='cuda:0', dtype=torch.float64), tensor(0.4800, device='cuda:0', dtype=torch.float64), tensor(0.5700, device='cuda:0', dtype=torch.float64), tensor(0.5300, device='cuda:0', dtype=torch.float64)]
Train Loss: 0.6832
Train Loss: 0.6832
Train Acc: 0.5300
Train Acc: 0.5300
Validation accuracy: [tensor(0.5726, device='cuda:0', dtype=torch.float64), tensor(0.5645, device='cuda:0', dtype=torch.float64), tensor(0.5565, device='cuda:0', dtype=torch.float64), tensor(0.4435, device='cuda:0', dtype=torch.float64), tensor(0.4839, device='cuda:0', dtype=torch.float64)]
Validation accuracy: [tensor(0.5726, device='cuda:0', dtype=


2019-05-10 03:52:40 Uploading - Uploading generated training modelTrain accuracy: [tensor(0.5200, device='cuda:0', dtype=torch.float64), tensor(0.5400, device='cuda:0', dtype=torch.float64), tensor(0.4800, device='cuda:0', dtype=torch.float64), tensor(0.5700, device='cuda:0', dtype=torch.float64), tensor(0.5300, device='cuda:0', dtype=torch.float64), tensor(0.4700, device='cuda:0', dtype=torch.float64), tensor(0.5900, device='cuda:0', dtype=torch.float64), tensor(0.5200, device='cuda:0', dtype=torch.float64), tensor(0.6800, device='cuda:0', dtype=torch.float64), tensor(0.6000, device='cuda:0', dtype=torch.float64)]
Train accuracy: [tensor(0.5200, device='cuda:0', dtype=torch.float64), tensor(0.5400, device='cuda:0', dtype=torch.float64), tensor(0.4800, device='cuda:0', dtype=torch.float64), tensor(0.5700, device='cuda:0', dtype=torch.float64), tensor(0.5300, device='cuda:0', dtype=torch.float64), tensor(0.4700, device='cuda:0', dtype=torch.float64), tensor(0.5900, device='cuda:0', dty


2019-05-10 03:52:56 Completed - Training job completed
Billable seconds: 150
